### Algebraic formulation of the problem

$$\begin{array}{rll}
 \text{max} & 7T+5C \\
 \text{s.t.} & 3T + 4C \le 2400 \\
 & 2T + C \le 1000 \\
  &  C \le 450 \\
   & T  \ge 100 \\
 & \text{non-negativity constraints}\\
\end{array}
$$

### 1. Slove for opimal objective value

### Set up Gurobi environment

In [1]:
from gurobipy import *

In [2]:
m = Model()

### Set up variable and constraint Python lists

In [3]:
products = ["tables", "chairs"]
constraints = ["carpentry_work", "painting"]

### Set  objective coefficients using Python dictionary

In [4]:
profit_contribution = {"tables": 7, "chairs": 5}

### Load variables into the Gurobi model

In [5]:
production_levels = m.addVars(products, name="production_levels")

### Inidicate maximum and minimum production level on each product

In [6]:
# set upper bounds on variables
production_levels["chairs"].ub = 450
# set lower bounds on variables
production_levels["tables"].lb = 100
m.update() # update Gurobi model

### Set the matrix of left-hand-side constraint coefficients (Python dictionary)

In [7]:
lhs_constr_matrix = {
    "carpentry_work": {"tables": 3, "chairs": 4},
    "painting":{"tables": 2, "chairs": 1}
}

### Set the right-hand-side constraint vector (Python dictionary)

In [8]:
rhs_constr_vector = {
    "carpentry_work": 2400,
    "painting": 1000
}

### Load the constraints into Gurobi. The use of quicksum and the for loops is convenient when there are many constraints.

In [9]:
constraints_set = m.addConstrs((quicksum(lhs_constr_matrix[constr][product]*production_levels[product] 
                      for product in products) <= rhs_constr_vector[constr]  # use sumproduct to generate one constraint
                      for constr in constraints), name="constraints");           # use for loop to generate all the constraints

### Set up the objective function

In [10]:
obj = quicksum(
    profit_contribution[product] * production_levels[product] for product in products
)

### Tell Gurobi we want to maximize this objective function

In [11]:
m.setObjective(obj, GRB.MAXIMIZE)

### Tell Gurobi to solve the LP, and look for the optimal objective function value on the last line

In [12]:
m.optimize()

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 7e+00]
  Bounds range     [1e+02, 4e+02]
  RHS range        [1e+03, 2e+03]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+31   1.750000e+30   2.800000e+01      0s
       2    4.0400000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  4.040000000e+03


### Ask Gurobi for the optimal decisions

In [13]:
m.printAttr('X')


    Variable            X 
-------------------------
production_levels[tables]          320 
production_levels[chairs]          360 


###  2. Sensitivity Analysis

### Set up variables x1, x2 in Python which refer to the variables in the model

In [14]:
x1 = m.getVars()[0]   # In Python, index 0 refers to the first element.So, x1 is tables, x2 is chairs
x2 = m.getVars()[1]

### Derive the maximum and minimum value that the objective coefficient of x1 can take before the optimal solution changes.

In [15]:
(x1.SAObjLow, x1.SAObjUp)

(3.75, 10.0)

This indicates that as long as the profit contribution of one table falls between (3.75, 10.0) interval, the optimal solution remains the same. Note that this is valid only when one coefficient changes.

### Similiarly, for x2

In [16]:
(x2.SAObjLow, x2.SAObjUp)

(3.5, 9.333333333333334)

#### As long as the profit contribution of one chair falls between (3.5, 9.34) interval, the optimal solution remains the same.

### Get the shadow price for contraint carpentry_work 

In [17]:
print(constraints_set['carpentry_work'].pi)

0.6


In [18]:
constraints_set['carpentry_work'].pi

0.6

The maximium to pay for one additional hour of carpentry work 0.6. 

In [19]:
print(constraints_set['painting'].pi)

2.6


The maximium to pay for one additional hour of painting is 2.6. 

### The maximum and minimum the RHS can take before the optimal solution changes.

In [20]:
print(constraints_set['carpentry_work'].SARHSLow,constraints_set['carpentry_work'].SARHSUp)

1500.0 2625.0


As long as the total hours available for carpentry work falls between (1500, 2625) interval, the optimal solution remains the same. Note that this is valid only when one constaint changes.

In [21]:
print(constraints_set['painting'].SARHSLow,constraints_set['painting'].SARHSUp)

850.0 1600.0


As long as the total hours available for painting falls between (850, 1600) interval, the optimal solution remains the same.

###   

###  3. Dual Linear Program

### algebraic formulation of the dual LP
$$\begin{array}{rll}
 \text{min} & 2400y1 + 1000y2 + 450y3 - 100y4 \\
 \text{s.t.} & 3y1 + 2y2 - y4\ge 7 \\
 & 4y1 + y2 + y3 \ge 5 \\
 & \text{non-negativity constraints}\\
\end{array}
$$

In [22]:
m_dual = Model()

### Set up variables and constraints using Python lists

In [23]:
dual_variables = ["carpentry_work_dual", "painting_dual", "max_chairs_dual", "min_tables_dual"]
dual_constraints = ["tables", "chairs"]

### Set  objective coefficients using Python dictionary

In [24]:
dual_contribution = {"carpentry_work_dual":2400, "painting_dual":1000, "max_chairs_dual":450, "min_tables_dual":-100}

### Load variables into the Gurobi model

In [25]:
dual_levels = m_dual.addVars(dual_variables, name="dual_levels")

In [26]:
for d in dual_variables:
    dual_levels[d].vtype = GRB.CONTINUOUS

### Set the matrix of left-hand-side constraint coefficients

In [27]:
dual_lhs_constr_matrix = {
    "tables": {"carpentry_work_dual": 3, "painting_dual": 2, "min_tables_dual": -1},
    "chairs":{"carpentry_work_dual": 4, "painting_dual": 1, "max_chairs_dual": 1}
}

### Set the right-hand-side constraint vector

In [28]:
dual_rhs_constr_vector = {
    "tables": 7,
    "chairs": 5
}

### Load the constraints into Gurobi. 

In [29]:
dual_constraints_set = m_dual.addConstrs((quicksum(dual_lhs_constr_matrix[constr][dual_variable] * dual_levels[dual_variable] 
                                                   for dual_variable in dual_lhs_constr_matrix[constr]) >= dual_rhs_constr_vector[constr] 
                                          for constr in dual_constraints), name="dual_constraints");

### Set up the objective function

In [30]:
dual_obj = quicksum(
    dual_contribution[dual_variable] * dual_levels[dual_variable] for dual_variable in dual_variables
)

### Tell Gurobi we want to minimize this objective function

In [31]:
m_dual.setObjective(dual_obj, GRB.MINIMIZE)

### Tell Gurobi to solve the dual LP, and look for the optimal objective function value on the last line

In [32]:
m_dual.optimize()

Optimize a model with 2 rows, 4 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 7e+00]
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+32   1.000000e+30   2.000000e+02      0s
       3    4.0400000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  4.040000000e+03


### Ask Gurobi for the optimal decisions

In [33]:
m_dual.printAttr('X')


    Variable            X 
-------------------------
dual_levels[carpentry_work_dual]          0.6 
dual_levels[painting_dual]          2.6 


In [34]:
m_dual.write("file.lp")

###  Sensitivity Analysis for Dual LP

#### Set up variables y1, y2, y3, y4 in Python which refer to the variables in the model

In [35]:
y1 = m_dual.getVars()[0]
y2 = m_dual.getVars()[1]
y3 = m_dual.getVars()[2]
y4 = m_dual.getVars()[3]

In [36]:
print(y1.X)
print(y2.X)
print(y3.X)
print(y4.X)

0.6
2.6
0.0
0.0


#### Derive the maximum and minimum value that the objective coefficient of y1, y2, y3, y4 can take before the optimal solution changes.

In [37]:
print ("carpentry_work_dual range:")
(y1.SAObjLow, y1.SAObjUp)

carpentry_work_dual range:


(1500.0, 2625.0)

In [38]:
print ("painting_dual range:")
(y2.SAObjLow, y2.SAObjUp)

painting_dual range:


(850.0, 1600.0)

In [39]:
print ("max_chairs_dual range:")
(y3.SAObjLow, y3.SAObjUp)

max_chairs_dual range:


(360.0, 1e+100)

In [40]:
print ("min_table_dual range:")
(y4.SAObjLow, y4.SAObjUp)

min_table_dual range:


(-320.0, 1e+100)

### Get the shadow price for contraint tables, chairs (primal constraints)

In [41]:
dual_constraints_set['tables'].pi

320.0

In [42]:
dual_constraints_set['chairs'].pi

360.0

### The maximum and minimum the RHS can take before the optimal solution changes.

In [43]:
print(dual_constraints_set['tables'].SARHSLow,dual_constraints_set['tables'].SARHSUp)


3.75 10.0


In [44]:
print(dual_constraints_set['chairs'].SARHSLow,dual_constraints_set['chairs'].SARHSUp)


3.5 9.333333333333334
